In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
dataset = load_dataset("emotion")

# Extracting features and labels
texts = dataset['train']['text']
labels = dataset['train']['label']

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to ensure uniform length
max_sequence_length = 100  # Adjust according to your data and requirements
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to one-hot encoding
label_dict = {label: i for i, label in enumerate(set(labels))}
num_classes = len(label_dict)
labels = [label_dict[label] for label in labels]
labels = tf.keras.utils.to_categorical(labels, num_classes)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16),  # Removed input_length
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))


No config specified, defaulting to: emotion/split
Found cached dataset emotion (/Users/rikigodani/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10


2024-05-05 15:53:50.848298: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


200/200 [==============================] - 7s 27ms/step - loss: 1.5492 - accuracy: 0.3795 - val_loss: 1.3243 - val_accuracy: 0.5397
Epoch 2/10
200/200 [==============================] - 5s 25ms/step - loss: 1.0682 - accuracy: 0.6188 - val_loss: 0.9933 - val_accuracy: 0.6403
Epoch 3/10
200/200 [==============================] - 5s 25ms/step - loss: 0.7850 - accuracy: 0.7225 - val_loss: 0.8732 - val_accuracy: 0.6947
Epoch 4/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5708 - accuracy: 0.8048 - val_loss: 0.7955 - val_accuracy: 0.7306
Epoch 5/10
200/200 [==============================] - 5s 26ms/step - loss: 0.3888 - accuracy: 0.8788 - val_loss: 0.6701 - val_accuracy: 0.7825
Epoch 6/10
200/200 [==============================] - 5s 26ms/step - loss: 0.2841 - accuracy: 0.9132 - val_loss: 0.6397 - val_accuracy: 0.7931
Epoch 7/10
200/200 [==============================] - 5s 26ms/step - loss: 0.2245 - accuracy: 0.9333 - val_loss: 0.6041 - val_accuracy: 0.8169
Epoch 8/10

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a function for making predictions
def predict_emotion(input_text):
    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # Make prediction
    prediction = model.predict(padded_sequence)
    
    # Decode the prediction
    emotion_labels = ['joy', 'anger', 'love', 'sadness', 'fear', 'surprise']
    predicted_label = emotion_labels[np.argmax(prediction)]
    prediction_value = np.max(prediction)
    
    return predicted_label, prediction_value

# Now, you can use this function to get predictions for any input text
input_text = input("Enter a text: ")
predicted_emotion, confidence = predict_emotion(input_text)
print("Predicted Emotion:", predicted_emotion)
print("Confidence:", confidence)
